In [4]:
from bs4 import BeautifulSoup
import lxml #is a parser for HTMl
import webscraping
import requests
import os
import re
import csv
#from selenium import webdriver
from langdetect import detect
import pandas as pd

We shoul generalize these functions... in webscraping.py!

## **[RQ1.1] Get the list of books**

We start to scrape the three pages to extract the URLs for each of them

In [23]:
UrlsFiles = open("urlpages.txt", "w")

for i in range(1, 101):
    url = "https://www.goodreads.com/list/show/1.Best_Books_Ever?page="+str(i)
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, features='lxml')
    for a in soup.find_all('a', class_="bookTitle"):
        UrlsFiles.write(a.get('href')+'\n')

UrlsFiles.close()

## **RQ[1.2] Crawl books**

In [24]:
headpart = "https://www.goodreads.com"

In [25]:
direct = "htmlpages" #Directory  
parent_dir = "./" #Parent Directory path 
pathAncestor = os.path.join(parent_dir, direct) #Path
os.mkdir(pathAncestor) #create the folder in the path

In [26]:
for i in range(1,101):
    os.makedirs(os.path.join(pathAncestor, 'page ' + str(i)))

In [34]:
UrlsFiles = open("urlpages.txt", "r")

counter_pages = 0
counter_html = 0
for x in UrlsFiles:
    if counter_html % 100 == 0:
        counter_pages = counter_pages+1
        
    counter_html = counter_html + 1
    
    subdirectory = pathAncestor + "/page " + str(counter_pages)
    article_name = "/article_"+str(counter_html)+".html"
    
    complete_path = subdirectory + article_name
    with open(complete_path, "wb") as ip_file:
        link = headpart + x
        try:
            page = requests.get(link)
        except:
            with open("failureRequest.txt", "wb") as err_file:
                err_file.write(link)
                err_file.close()
        
        soup = BeautifulSoup(page.text, features='lxml')
        
        ip_file.write(soup.encode('utf-8'))
        ip_file.close()

UrlsFiles.close()

## **RQ[1.3] Parse downloaded pages**

In [20]:
def scrap_book(tsv_writer, article):
    global bookTitle, bookSeries, bookAuthors, ratingValue, ratingCount, reviewCount, Plot, NumberofPages, Published, Characters, Setting, Url;
    
    with open(article, 'r', encoding="utf-8") as out_file:
        contents = out_file.read()
        soup = BeautifulSoup(contents, features="lxml") #parse the text

        #extract rating and review count
        try:
            ratings = soup.find_all('a', href="#other_reviews") #search the ratings in its
            rating_count = -1
            rating = -1
            for raiting in ratings:
                if raiting.find_all('meta', itemprop="ratingCount"):
                    ratingCount = raiting.text.replace('\n', '').strip().split(' ')[0].replace(',', '')
                elif raiting.find_all('meta', itemprop="reviewCount"):
                    reviewCount = raiting.text.replace('\n', '').strip().split(' ')[0].replace(',', '')
        except:
            ratingCount = " "
            reviewCount = " "
            
        #extract the book title
        try:
            bookTitle = soup.find_all('h1')[0].contents[0].replace('\n', '').strip()
        except:
            bookTitle = " "

        #extract the book authors
        try:
            bookAuthors = soup.find_all('span', itemprop='name')[0].contents[0]
        except:
            bookAuthors = " "

        #extract the book authors, we shoul FIX it.
        try:
            Plot = soup.find_all('div', id="description")[0].contents[3].text
            if detect(Plot) != "en":
                Plot = " "
        except:
            try:
                Plot = soup.find_all('div', id="description")[0].contents[1].text
                if detect(Plot) != "en":
                    Plot = " "
            except:
                Plot = " "
                

        #extract the date
        try:
            date = soup.find_all('div', id="details")[0].contents[3].text.replace('\n', '').strip().split()
            Published = date[1]+" "+date[2]+" "+date[3]
        except:
            Published = " "

        #Rating Value
        try:
            ratingValue = soup.find('span', itemprop="ratingValue").text.strip()
        except:
            ratingValue = " "

        #Number of pages
        try:
            NumberofPages = soup.find('span', itemprop="numberOfPages").text.split()[0]
        except:
            NumberofPages = " "

        #Title series
        try:
            bookSeries = soup.find_all('a', href= re.compile(r'/series/*'))[0].contents[0].strip()
        except:
            bookSeries = " "
            
        #Places
        try:
            Setting = []
            for places in soup.find_all('a', href= re.compile(r'/places/*')):
                Setting.append(places.text)
            Setting = ", ".join(Setting) if len(Setting)>=1 else " "
        except:
            Setting = " "

        #list of characters
        try:
            Characters = []
            for character in soup.find_all('a', href= re.compile(r'/characters/*') ):
                Characters.append(character.text)
            Characters = ", ".join(Characters) if len(Characters)>=1 else " "
        except:
            Characters = " "

        #extract the Url
        try:
            Url = soup.find_all('link')[0]["href"]
        except:
            Url = " "

        tsv_writer.writerow([bookTitle, bookSeries, bookAuthors, ratingValue, ratingCount, reviewCount, Plot, NumberofPages, Published, Characters, Setting, Url])

In [ ]:
path = "./html_pages"

filenames = os.listdir(path)
for i in range(4, 11):
    filenames = os.listdir(path + '/' + str(i))

    for file in filenames:
        with open(path + '/' + str(i) + './article_'+str(file.split("_")[1].replace(".html", ""))+'.tsv', 'w', encoding="utf-8", newline='') as out_file:
            tsv_writer = csv.writer(out_file, delimiter='\t')
            tsv_writer.writerow(['bookTitle', 'bookSeries', 'bookAuthors', 'ratingValue', 
                            'ratingCount', 'reviewCount', 'Plot', 'NumberofPages', 'Published',
                            'Characters', 'Setting', 'Url'])
            scrap_book(tsv_writer, path + '/' + str(i) + "/" + file)

## RQ[2] Search Engine

In [13]:
df1 = pd.read_csv('NEWoutput1.tsv',sep="\t")
df2 = pd.read_csv('NEWoutput2.tsv', sep="\t")
df3 = pd.read_csv('NEWoutput3.tsv', sep="\t")

complete_dataset = pd.concat([df1, df2, df3])
complete_dataset = complete_dataset.reset_index(drop=True) # reset index

complete_dataset = complete_dataset.drop_duplicates(subset=['bookTitle'])
complete_dataset = complete_dataset.dropna(subset=['Plot'])
complete_dataset = complete_dataset.reset_index(drop=True)
complete_dataset

,bookTitle,bookSeries,bookAuthors,ratingValue,ratingCount,reviewCount,Plot,NumberofPages,Published,Characters,Setting,Url
0,The Hunger Games,(The Hunger Games #1),Suzanne Collins,4.33,6413424,172616,"Could you survive on your own in the wild, wit...",374,September 14th 2008,"Katniss Everdeen, Peeta Mellark, Cato (Hunger ...","District 12, Panem, Capitol, Panem, Panem",https://www.goodreads.com/book/show/2767052-th...
1,The Fault in Our Stars,,John Green,4.20,3575591,155851,Despite the tumor-shrinking medical miracle th...,313,January 10th 2012,"Hazel Grace Lancaster, Augustus Waters, Isaac","Indianapolis, Indiana, Amsterdam",https://www.goodreads.com/book/show/11870085-t...
2,A Prayer for Owen Meany,,John Irving,4.23,286719,13851,"Eleven-year-old Owen Meany, playing in a Littl...",637,1990 by Black,"Owen Meany, John Wheelwright","Gravesend, New Hampshire, Toronto, Ontario",https://www.goodreads.com/book/show/4473.A_Pra...
3,Gone with the Wind,,Margaret Mitchell,4.30,1078968,19913,"Scarlett O'Hara, the beautiful, spoiled daught...",1037,April 1st 1999,"Scarlett O'Hara, Rhett Butler, Ashley Wilkes, ...","Atlanta, Georgia",https://www.goodreads.com/book/show/18405.Gone...
4,The Hitchhiker's Guide to the Galaxy,(The Hitchhiker's Guide to the Galaxy #1),Douglas Adams,4.22,1446923,30393,Seconds before the Earth is demolished to make...,193,June 23rd 2007,"Zaphod Beeblebrox, Arthur Dent, Ford Prefect, ...",,https://www.goodreads.com/book/show/386162.The...
...,...,...,...,...,...,...,...,...,...,...,...,...
25064,Daddy's Girl,NaN,Debbie Drechsler,3.75,357,50,Daddy's Girl is a powerful book that uses a ch...,79,December 31st 1996,NaN,NaN,https://www.goodreads.com/book/show/530449.Dad...
25065,Rude Buay ... The Unstoppable,(Rude Buay #1),John A. Andrews,4.23,22,2,"Author John A. Andrews, son of the Caribbean s...",220,July 15th 2010,NaN,NaN,https://www.goodreads.com/book/show/10648451-r...
25066,Shrub: The Short but Happy Political Life of G...,NaN,Molly Ivins,4.09,2236,88,"When it comes to reporting on politics, nobody...",193,October 10th 2000,NaN,NaN,https://www.goodreads.com/book/show/169710.Shrub
25067,"My Story as told by Water: Confessions, Druidi...",NaN,David James Duncan,4.11,798,65,"In this remarkable collection of essays, David...",304,July 17th 2001,NaN,NaN,https://www.goodreads.com/book/show/23194.My_S...


## **Cleaning data**

In [48]:
#DEF Cleaning
import nltk

from nltk.corpus import stopwords
from nltk.stem.snowball import PorterStemmer
from nltk.tokenize import RegexpTokenizer
#from nltk.stem import WordNetLemmatizer

In [49]:
def cleaningDataset(df):
    for i, row in df.iterrows():
        tokenizer = RegexpTokenizer("[\w']+")  # import the tokenizer punctuation
        
        df.at[i, 'Plot'] = tokenizer.tokenize(df.at[i, 'Plot'].lower()) # remove the punctuation
        df.at[i, 'Plot'] = [w for w in df.at[i, 'Plot'] if not w in set(stopwords.words('english'))]  # words in english to avoid few data for the cleaning data
        df.at[i, 'Plot'] = [PorterStemmer().stem(word) for word in df.at[i, 'Plot']] # contesto
    return df

In [5]:
def createVocabulary(df):
    vocabulary = {}
    for i, row in df.iterrows():
        for word in df.at[i, 'Plot']:
            if word in vocabulary.keys():
                if "document_"+str(i) not in vocabulary[word]:
                    vocabulary[word].append("document_"+str(i))
            else:
                vocabulary[word] = ["document_"+str(i)]
    return vocabulary

In [80]:
#create and clean the dataset
df = complete_dataset.copy()
df = cleaningDataset(df)

vocabulary = createVocabulary(df)    

In [47]:
#create file .csv
import csv

with open('./inverted_list.tsv', 'w', encoding="utf-8", newline='') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(["Word", "Term_id", "Document_List"])
    
    list_of_words = vocabulary.keys()
    for id, word in enumerate(list_of_words, 1):
        term_id_i = "term_id_"+str(id)
        tsv_writer.writerow([word, term_id_i, vocabulary[word]]) 

## RQ[2.1.2] Execute the query

In [4]:
query = input("Insert your query string: ")

Insert your query string: Survival games


viva la patacca

In [12]:
stop = set(stopwords.words('english')) # words in english to avoid few data for the cleaning data
stemmer = PorterStemmer() # contesto

def cleanQuery(row):
    row = row.split(" ") # split the string query
    for element in row:
        tokenizer = RegexpTokenizer("[\w']+")  # import the tokenizer punctuation
        element = tokenizer.tokenize(element.lower()) # remove the punctuation
    
    row = [w for w in row if not w in stop]
    
    #row = [lemmatizer.lemmatize(word) for word in row]
    row = [stemmer.stem(word) for word in row]
    
    return row

In [13]:
cleanQString = cleanQuery(query)
cleanQString

['surviv', 'game']

open the dataset vocabulary

In [6]:
inv_lst = pd.read_csv('inverted_list.tsv',sep="\t")

inv_lst = inv_lst.drop_duplicates(subset=['Word'])
inv_lst = inv_lst.reset_index(drop=True)

In [7]:
inv_lst.head()

,Word,Term_id,Document_List
0,could,term_id_1,"['document_1', 'document_27', 'document_39', '..."
1,surviv,term_id_2,"['document_1', 'document_27', 'document_41', '..."
2,wild,term_id_3,"['document_1', 'document_9', 'document_20', 'd..."
3,everi,term_id_4,"['document_1', 'document_4', 'document_6', 'do..."
4,one,term_id_5,"['document_1', 'document_10', 'document_15', '..."


In [8]:
inv_lst.set_index('Word', inplace=True)

In [9]:
inv_lst.head()

,Term_id,Document_List
Word,,
could,term_id_1,"['document_1', 'document_27', 'document_39', '..."
surviv,term_id_2,"['document_1', 'document_27', 'document_41', '..."
wild,term_id_3,"['document_1', 'document_9', 'document_20', 'd..."
everi,term_id_4,"['document_1', 'document_4', 'document_6', 'do..."
one,term_id_5,"['document_1', 'document_10', 'document_15', '..."


In [18]:
document_lists = inv_lst.loc[cleanQString, "Document_List"]

In [19]:
document_lists

Word
surviv    ['document_1', 'document_27', 'document_41', '...
game      ['document_1', 'document_3', 'document_21', 'd...
Name: Document_List, dtype: object

In [43]:
def make_it_string(word):
    for sym in ["[", "]", "'"]:
        word = word.replace(sym, "")
    word = word.split(", ")
    
    return word

In [20]:
intersection_list = []
for word in document_lists:
    word = make_it_string(word)

    if not intersection_list:
        intersection_list = list(word)
    else:
        intersection_list = set(intersection_list).intersection(list(word))

In [21]:
intersection_list

{'document_1',
 'document_10437',
 'document_11031',
 'document_11126',
 'document_11979',
 'document_12042',
 'document_12624',
 'document_12911',
 'document_12922',
 'document_12995',
 'document_13088',
 'document_13348',
 'document_13628',
 'document_13940',
 'document_14132',
 'document_14139',
 'document_14645',
 'document_1465',
 'document_14658',
 'document_14731',
 'document_14838',
 'document_1524',
 'document_15796',
 'document_16922',
 'document_17152',
 'document_17440',
 'document_17613',
 'document_18261',
 'document_18319',
 'document_18493',
 'document_18884',
 'document_19208',
 'document_19610',
 'document_19615',
 'document_20441',
 'document_20491',
 'document_20561',
 'document_20729',
 'document_21021',
 'document_21365',
 'document_21619',
 'document_22117',
 'document_222',
 'document_22295',
 'document_22311',
 'document_2240',
 'document_22495',
 'document_22586',
 'document_22790',
 'document_23082',
 'document_23367',
 'document_23448',
 'document_23658',
 '

In [22]:
#show documents
for num in intersection_list:
    num = num.split("_")[1]
    print(complete_dataset.loc[int(num)-1, ["bookTitle", "Plot", "Url"]])

bookTitle                                     The War of Mists
Plot         She chose to die in her sister's place and yet...
Url          https://www.goodreads.com/book/show/6655491-th...
Name: 8035, dtype: object
bookTitle                                         Wicked Games
Plot         Abby Lewis never pictured herself on the survi...
Url          https://www.goodreads.com/book/show/10719342-w...
Name: 21618, dtype: object
bookTitle                                       Secret Society
Plot         MIASHA'S STUNNING DEBUT NOVEL... WITH A TWIST ...
Url          https://www.goodreads.com/book/show/184999.Sec...
Name: 22585, dtype: object
bookTitle                                  A Child Called "It"
Plot         Also see: Alternate Cover Editions for this IS...
Url          https://www.goodreads.com/book/show/60748.A_Ch...
Name: 373, dtype: object
bookTitle                                     The Dark Calling
Plot         New intrigues, adventures, and stunning revela...
Url          

Name: 23819, dtype: object
bookTitle                                           Anvil Gate
Plot         Continuing the saga of the bestselling game se...
Url          https://www.goodreads.com/book/show/7289702-an...
Name: 23447, dtype: object
bookTitle                             Mass Effect: Retribution
Plot         Humanity has reached the stars, joining the va...
Url          https://www.goodreads.com/book/show/7552700-ma...
Name: 13627, dtype: object
bookTitle                                                 U-10
Plot         The U-10 is a survival kit you might not survi...
Url          https://www.goodreads.com/book/show/51106657-u-10
Name: 3915, dtype: object
bookTitle                                     Cage of Darkness
Plot         While traveling to Fren, Allyssa and Odar are ...
Url          https://www.goodreads.com/book/show/33893388-c...
Name: 11030, dtype: object
bookTitle                                             Day Zero
Plot         Arcana means secrets, and these Ar

Name: 7397, dtype: object
bookTitle                                       Ender's Shadow
Plot         Welcome to Battleschool.Growing up is never ea...
Url          https://www.goodreads.com/book/show/9532.Ender...
Name: 709, dtype: object
bookTitle    The Wilderness Family: At Home with Africa's W...
Plot         Everyone warned Kobie Krüger that being the wi...
Url          https://www.goodreads.com/book/show/830265.The...
Name: 16921, dtype: object
bookTitle                                          In the Dark
Plot         A secluded mountain lodge. The perfect getaway...
Url          https://www.goodreads.com/book/show/44527037-i...
Name: 12623, dtype: object
bookTitle                                                Truth
Plot         From New York Times and USA Today bestselling ...
Url          https://www.goodreads.com/book/show/16070018-t...
Name: 7674, dtype: object
bookTitle                                       Highland Lover
Plot         HIGHLAND DANGEREver inquisitive, Lady

## RQ[2.2] Conjunctive query & Ranking score

In [146]:
#create the second inverted list
inv_lst = pd.read_csv('inverted_list.tsv',sep="\t")
inv_lst.head()

,Word,Term_id,Document_List
0,could,term_id_1,"['document_1', 'document_27', 'document_39', '..."
1,surviv,term_id_2,"['document_1', 'document_27', 'document_41', '..."
2,wild,term_id_3,"['document_1', 'document_9', 'document_20', 'd..."
3,everi,term_id_4,"['document_1', 'document_4', 'document_6', 'do..."
4,one,term_id_5,"['document_1', 'document_10', 'document_15', '..."


In [147]:
#create and clean the dataset
df = complete_dataset.copy()
df = cleaningDataset(df)

In [148]:
inv_lst['Document_PairList'] = ""

In [149]:
inv_lst

,Word,Term_id,Document_List,Document_PairList
0,could,term_id_1,"['document_1', 'document_27', 'document_39', '...",
1,surviv,term_id_2,"['document_1', 'document_27', 'document_41', '...",
2,wild,term_id_3,"['document_1', 'document_9', 'document_20', 'd...",
3,everi,term_id_4,"['document_1', 'document_4', 'document_6', 'do...",
4,one,term_id_5,"['document_1', 'document_10', 'document_15', '...",
...,...,...,...,...
66496,dubya',term_id_66497,['document_25067'],
66497,'git,term_id_66498,['document_25067'],
66498,tuff',term_id_66499,['document_25067'],
66499,commi,term_id_66500,['document_25067'],


In [151]:
import math

for i, row in inv_lst.iterrows():
    lst_doc = make_it_string(inv_lst.at[i, 'Document_List'])
    for doc in lst_doc:
        try:
            number_doc = int(doc.split("_")[1])

            interested_row = df.at[number_doc, "Plot"]

            tf = interested_row.count(inv_lst.at[i, "Word"]) / len(interested_row)

            idf = math.log(len(df)/len(doc))

            tf_idf = round(tf * idf, 3)

            if not inv_lst.at[i, "Document_PairList"]:
                inv_lst.at[i, "Document_PairList"] = [(doc, tf_idf)]
            else:
                inv_lst.at[i, "Document_PairList"] = inv_lst.at[i, "Document_PairList"].append((doc, tf_idf)) 
        except:
            print(doc)
inv_lst

document_1013
document_1146
document_1395
document_2049
document_2400
document_2607
document_2651
document_2819
document_2981
document_3021
document_3184
document_3291
document_3923
document_3989
document_4048
document_4237
document_4528
document_4635
document_4746
document_5274
document_5469
document_5692
document_5721
document_5777
document_5838
document_5866
document_6604
document_6878
document_6898
document_6903
document_6922
document_6926
document_7061
document_7209
document_7254
document_7415
document_7471
document_7903
document_8066
document_8143
document_8181
document_8335
document_8466
document_8604
document_8697
document_9381
document_10595
document_10689
document_10777
document_10843
document_10932
document_10934
document_11155
document_11175
document_11254
document_11471
document_11674
document_11684
document_11866
document_11895
document_12127
document_12167
document_12214
document_12263
document_12300
document_12330
document_12366
document_12606
document_12747
document_13

document_489
document_1013
document_1015
document_1395
document_1471
document_1666
document_1702
document_1734
document_1763
document_1905
document_1907
document_2400
document_2560
document_2587
document_3018
document_3021
document_3291
document_3527
document_3544
document_3689
document_3829
document_3841
document_3923
document_3986
document_3989
document_4048
document_4429
document_4528
document_4672
document_4712
document_4958
document_5197
document_5312
document_5426
document_5612
document_5734
document_5801
document_5838
document_6279
document_6415
document_6709
document_6775
document_6959
document_6961
document_6995
document_7125
document_7156
document_7179
document_7328
document_7342
document_7428
document_7646
document_8040
document_8103
document_8130
document_8251
document_8335
document_8392
document_8462
document_8548
document_8565
document_8604
document_8745
document_8782
document_8920
document_9035
document_9217
document_9236
document_9349
document_9441
document_9575
documen

document_1476
document_1702
document_1732
document_1763
document_2596
document_2996
document_3123
document_3343
document_3766
document_3829
document_3923
document_3934
document_3986
document_3989
document_4635
document_5535
document_5729
document_6721
document_6786
document_6866
document_6903
document_7061
document_8040
document_8066
document_8143
document_8335
document_8372
document_8379
document_8584
document_8651
document_8795
document_9022
document_9035
document_9246
document_9719
document_9808
document_10087
document_10151
document_10193
document_10251
document_10496
document_10580
document_10689
document_11054
document_11060
document_11073
document_11155
document_11199
document_11222
document_11280
document_11471
document_11484
document_12050
document_12082
document_12174
document_12217
document_12271
document_12278
document_12285
document_12366
document_12394
document_12437
document_12698
document_12963
document_13102
document_13111
document_13194
document_13209
document_13252
d

document_17916
document_17966
document_17970
document_17982
document_18030
document_18058
document_18291
document_25069
document_3108
document_4093
document_4528
document_4880
document_4974
document_5085
document_7057
document_7061
document_7428
document_8024
document_8040
document_8168
document_8411
document_8554
document_8565
document_8734
document_9236
document_9254
document_9298
document_10151
document_10319
document_10564
document_10580
document_11060
document_11118
document_11254
document_11469
document_11517
document_11786
document_11926
document_12029
document_12285
document_12309
document_12366
document_12556
document_13356
document_14108
document_14147
document_15883
document_17185
document_17346
document_17386
document_17389
document_17646
document_17790
document_17836
document_18177
document_1382
document_3123
document_3766
document_3923
document_4334
document_4429
document_4712
document_4974
document_5612
document_5692
document_5914
document_6541
document_7428
document_798

document_1702
document_2338
document_2590
document_3018
document_3812
document_3989
document_5838
document_6922
document_7254
document_7328
document_7677
document_8066
document_8111
document_8554
document_8584
document_8604
document_8697
document_8734
document_8828
document_9130
document_9441
document_9674
document_10042
document_10304
document_10568
document_10896
document_11118
document_11222
document_11239
document_11421
document_11471
document_11895
document_12517
document_13063
document_13140
document_13187
document_13252
document_13610
document_13822
document_13915
document_13983
document_14028
document_14412
document_14859
document_14932
document_15328
document_15544
document_15909
document_16412
document_16441
document_16933
document_17216
document_17300
document_17672
document_17819
document_17860
document_17916
document_18030
document_1956
document_2766
document_4053
document_4093
document_4528
document_6543
document_6588
document_7342
document_7903
document_8143
document_857

...

document_14408
document_7297
document_11538
document_17389
document_5729
document_13140
document_10843
document_12606
document_13364
document_8604
document_12274
document_14798
document_14901
document_11743
document_15620
document_5998
document_4393
document_12112
document_17672
document_12223
document_10042
document_7265
document_14912
document_11528
document_7309
document_6543
document_4429
document_4429
document_4429
document_4429
document_4429
document_4429
document_9653
document_7125
document_16615
document_16933
document_8697
document_10135
document_8335
document_6818
document_6818
document_6818
document_11674
document_12033
document_16042
document_17685
document_12121
document_13533
document_9289
document_9674
document_11528
document_4491
document_4491
document_4491
document_4491
document_17862
document_13530
document_9954
document_4958
document_4958
document_13328
document_10843
document_17230
document_5535
document_12244
document_18291
document_5409
document_13296
document_162

document_17386
document_16042
document_11054
document_14801
document_7105
document_14461
document_7125
document_7125
document_7125
document_7125
document_16128
document_13543
document_17185
document_7156
document_8560
document_16799
document_7171
document_7171
document_7171
document_7171
document_7171
document_7179
document_7179
document_7179
document_7179
document_7179
document_7179
document_7179
document_7183
document_9298
document_7183
document_7183
document_7183
document_7183
document_7186
document_7186
document_7186
document_7186
document_7186
document_7186
document_7186
document_7186
document_7186
document_7186
document_7186
document_7186
document_12241
document_13460
document_7201
document_7201
document_10251
document_7209
document_17125
document_12609
document_12239
document_16847
document_18267
document_14427
document_11484
document_13915
document_8462
document_7254
document_11670
document_7265
document_7265
document_7265
document_17781
document_8795
document_8795
document_140

document_17436
document_15620
document_18199
document_11326
document_13230
document_18033
document_11670
document_11258
document_17862
document_9950
document_9954
document_9954
document_9954
document_9954
document_9954
document_9954
document_9954
document_13915
document_17860
document_15909
document_15909
document_15909
document_10042
document_10042
document_10149
document_10059
document_10059
document_10059
document_10059
document_10059
document_10059
document_10059
document_11696
document_11637
document_14257
document_10087
document_10087
document_10087
document_10087
document_10087
document_10087
document_10087
document_10087
document_10087
document_10087
document_10087
document_14285
document_17646
document_13194
document_11576
document_14979
document_14979
document_14979
document_10135
document_10135
document_10135
document_13738
document_10135
document_10135
document_13738
document_10135
document_10934
document_10149
document_18033
document_10149
document_10149
document_10149
doc

document_13724
document_13724
document_13724
document_13724
document_13724
document_13724
document_13724
document_13746
document_13746
document_13746
document_13777
document_13777
document_13777
document_13779
document_13800
document_13810
document_13822
document_13822
document_13822
document_13822
document_13822
document_13822
document_13822
document_13822
document_13831
document_13831
document_13862
document_13862
document_13862
document_13868
document_13868
document_13868
document_13868
document_13868
document_13868
document_13885
document_13896
document_13909
document_13911
document_13911
document_13915
document_13915
document_13915
document_14420
document_14420
document_13924
document_13924
document_13924
document_14285
document_13983
document_14412
document_15106
document_13997
document_14012
document_14012
document_14012
document_14014
document_14014
document_14018
document_14018
document_14018
document_14022
document_14028
document_14030
document_14030
document_14030
document_1

,Word,Term_id,Document_List,Document_PairList
0,could,term_id_1,"['document_1', 'document_27', 'document_39', '...","[(document_25067, 0.0)]"
1,surviv,term_id_2,"['document_1', 'document_27', 'document_41', '...","[(document_25062, 0.0)]"
2,wild,term_id_3,"['document_1', 'document_9', 'document_20', 'd...",None
3,everi,term_id_4,"['document_1', 'document_4', 'document_6', 'do...",None
4,one,term_id_5,"['document_1', 'document_10', 'document_15', '...","[(document_25067, 0.0)]"
...,...,...,...,...
66496,dubya',term_id_66497,['document_25067'],"[(document_25067, 0.0)]"
66497,'git,term_id_66498,['document_25067'],"[(document_25067, 0.0)]"
66498,tuff',term_id_66499,['document_25067'],"[(document_25067, 0.0)]"
66499,commi,term_id_66500,['document_25067'],"[(document_25067, 0.0)]"


In [ ]:
with open('./inverted_list_2.tsv', 'w', encoding="utf-8", newline='') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(["Word", "Term_id", "(Document_i, tfIdf)"])
    
    list_of_words = vocabulary.keys()
    for id, word in enumerate(list_of_words, 1):
        term_id_i = "term_id_"+str(id)
        tsv_writer.writerow([word, term_id_i, vocabulary[word]]) 